In [205]:
import pandas as pd
from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

Loading CSVs
==

In [193]:
d_city = pd.read_csv('../data/diamonds_city.csv')
d_clarity = pd.read_csv('../data/diamonds_clarity.csv')
d_color = pd.read_csv('../data/diamonds_color.csv')
d_cut = pd.read_csv('../data/diamonds_cut.csv')
d_dimensions = pd.read_csv('../data/diamonds_dimensions.csv')
d_properties = pd.read_csv('../data/diamonds_properties.csv')
d_transactional = pd.read_csv('../data/diamonds_transactional.csv')
d_test = pd.read_csv('../data/diamonds_test.csv')

In [8]:
d_city.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   city_id  13 non-null     object
 1   city     13 non-null     object
dtypes: object(2)
memory usage: 336.0+ bytes


In [9]:
d_clarity.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   clarity_id  8 non-null      object
 1   clarity     8 non-null      object
dtypes: object(2)
memory usage: 256.0+ bytes


In [10]:
d_color.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   color_id  7 non-null      object
 1   color     7 non-null      object
dtypes: object(2)
memory usage: 240.0+ bytes


In [11]:
d_cut.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cut_id  5 non-null      object
 1   cut     5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


In [12]:
d_dimensions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index_id  40455 non-null  object 
 1   depth     40455 non-null  float64
 2   table     40455 non-null  float64
 3   x         40455 non-null  float64
 4   y         40455 non-null  float64
 5   z         40455 non-null  float64
dtypes: float64(5), object(1)
memory usage: 1.9+ MB


In [13]:
d_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   index_id    40455 non-null  object
 1   cut_id      40455 non-null  object
 2   color_id    40455 non-null  object
 3   clarity_id  40455 non-null  object
dtypes: object(4)
memory usage: 1.2+ MB


In [14]:
d_transactional.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40455 entries, 0 to 40454
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index_id  40455 non-null  object 
 1   price     40455 non-null  int64  
 2   city_id   40455 non-null  object 
 3   carat     40455 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.2+ MB


In [76]:
d_test.count()

id         13485
carat      13485
cut        13485
color      13485
clarity    13485
depth      13485
table      13485
x          13485
y          13485
z          13485
city       13485
dtype: int64

In [208]:
diamonds = d_dimensions.merge(d_properties, how='inner', on='index_id')
diamonds = diamonds.merge(d_cut, how='inner', on='cut_id')
diamonds = diamonds.merge(d_color, how='inner', on='color_id')
diamonds = diamonds.merge(d_clarity, how='inner', on='clarity_id')
diamonds = diamonds.merge(d_transactional, how='inner', on='index_id')
diamonds = diamonds.merge(d_city, how='inner', on='city_id')
diamonds = diamonds.drop(columns=['cut_id', 'color_id', 'clarity_id', 'city_id'])
diamonds

,index_id,depth,table,x,y,z,cut,color,clarity,price,carat,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,Premium,J,VS2,4268,1.21,Kimberly
1,248aa2bdd0032920ac9e5f6ad36c350549da067efeaf7b...,60.8,60.0,6.85,6.89,4.18,Premium,J,VS2,4839,1.20,Kimberly
2,72b31cf00f8ab3967588fad4a32f61622cb162f9b7bc2c...,60.6,59.0,4.34,4.38,2.64,Premium,J,VS2,368,0.30,Kimberly
3,98c53df687f2e9b94da80eef5b9049f1fac456b4c41c80...,62.6,57.0,6.80,6.72,4.23,Premium,J,VS2,5053,1.20,Kimberly
4,5dfe43a321c6834c7de273c73aeadc705d919a5869e0f5...,59.4,62.0,6.66,6.58,3.93,Premium,J,VS2,3593,1.05,Kimberly
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,e03a231c5b52635043e7dc5f0c6c9f16722e14dbcc98bb...,61.8,56.0,4.42,4.46,2.74,Ideal,F,IF,978,0.33,Zurich
40451,90dcb905e13140ff99770039b843fb62fb179ab4a3bae9...,61.6,56.0,4.43,4.47,2.74,Ideal,F,IF,929,0.32,Zurich
40452,97d3c3344c245422ee7fa4f448b2cace9940121620df22...,62.4,55.0,4.20,4.17,2.61,Ideal,F,IF,828,0.28,Zurich
40453,e4dc4e0761ccc6fbb4c064517e40f3582522c325f9a4e5...,62.0,58.0,6.44,6.49,4.01,Ideal,F,IF,11116,1.02,Zurich


In [32]:
diamonds.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40455 entries, 0 to 40454
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index_id  40455 non-null  object 
 1   depth     40455 non-null  float64
 2   table     40455 non-null  float64
 3   x         40455 non-null  float64
 4   y         40455 non-null  float64
 5   z         40455 non-null  float64
 6   cut       40455 non-null  object 
 7   color     40455 non-null  object 
 8   clarity   40455 non-null  object 
 9   price     40455 non-null  int64  
 10  carat     40455 non-null  float64
 11  city      40455 non-null  object 
dtypes: float64(6), int64(1), object(5)
memory usage: 4.0+ MB


In [33]:
diamonds['city'].unique()

array(['Kimberly', 'London', 'Madrid', 'Antwerp', 'Surat', 'Tel Aviv',
       'New York City', 'Las Vegas', 'Luxembourg', 'Dubai', 'Paris',
       'Amsterdam', 'Zurich'], dtype=object)

In [209]:
diamonds

,index_id,depth,table,x,y,z,cut,color,clarity,price,carat,city
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,Premium,J,VS2,4268,1.21,Kimberly
1,248aa2bdd0032920ac9e5f6ad36c350549da067efeaf7b...,60.8,60.0,6.85,6.89,4.18,Premium,J,VS2,4839,1.20,Kimberly
2,72b31cf00f8ab3967588fad4a32f61622cb162f9b7bc2c...,60.6,59.0,4.34,4.38,2.64,Premium,J,VS2,368,0.30,Kimberly
3,98c53df687f2e9b94da80eef5b9049f1fac456b4c41c80...,62.6,57.0,6.80,6.72,4.23,Premium,J,VS2,5053,1.20,Kimberly
4,5dfe43a321c6834c7de273c73aeadc705d919a5869e0f5...,59.4,62.0,6.66,6.58,3.93,Premium,J,VS2,3593,1.05,Kimberly
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,e03a231c5b52635043e7dc5f0c6c9f16722e14dbcc98bb...,61.8,56.0,4.42,4.46,2.74,Ideal,F,IF,978,0.33,Zurich
40451,90dcb905e13140ff99770039b843fb62fb179ab4a3bae9...,61.6,56.0,4.43,4.47,2.74,Ideal,F,IF,929,0.32,Zurich
40452,97d3c3344c245422ee7fa4f448b2cace9940121620df22...,62.4,55.0,4.20,4.17,2.61,Ideal,F,IF,828,0.28,Zurich
40453,e4dc4e0761ccc6fbb4c064517e40f3582522c325f9a4e5...,62.0,58.0,6.44,6.49,4.01,Ideal,F,IF,11116,1.02,Zurich


Features
==

In [210]:
def super_feature(df):
    return df['depth'] * df['table']
diamonds['super_feature'] = super_feature(diamonds)
diamonds

,index_id,depth,table,x,y,z,cut,color,clarity,price,carat,city,super_feature
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,Premium,J,VS2,4268,1.21,Kimberly,3619.2
1,248aa2bdd0032920ac9e5f6ad36c350549da067efeaf7b...,60.8,60.0,6.85,6.89,4.18,Premium,J,VS2,4839,1.20,Kimberly,3648.0
2,72b31cf00f8ab3967588fad4a32f61622cb162f9b7bc2c...,60.6,59.0,4.34,4.38,2.64,Premium,J,VS2,368,0.30,Kimberly,3575.4
3,98c53df687f2e9b94da80eef5b9049f1fac456b4c41c80...,62.6,57.0,6.80,6.72,4.23,Premium,J,VS2,5053,1.20,Kimberly,3568.2
4,5dfe43a321c6834c7de273c73aeadc705d919a5869e0f5...,59.4,62.0,6.66,6.58,3.93,Premium,J,VS2,3593,1.05,Kimberly,3682.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,e03a231c5b52635043e7dc5f0c6c9f16722e14dbcc98bb...,61.8,56.0,4.42,4.46,2.74,Ideal,F,IF,978,0.33,Zurich,3460.8
40451,90dcb905e13140ff99770039b843fb62fb179ab4a3bae9...,61.6,56.0,4.43,4.47,2.74,Ideal,F,IF,929,0.32,Zurich,3449.6
40452,97d3c3344c245422ee7fa4f448b2cace9940121620df22...,62.4,55.0,4.20,4.17,2.61,Ideal,F,IF,828,0.28,Zurich,3432.0
40453,e4dc4e0761ccc6fbb4c064517e40f3582522c325f9a4e5...,62.0,58.0,6.44,6.49,4.01,Ideal,F,IF,11116,1.02,Zurich,3596.0


In [211]:
X = diamonds[['cut', 'color', 'clarity', 'carat', 'super_feature']]

y = diamonds['price']

Encoding
==

In [212]:
X_encoding = pd.get_dummies(X, columns=['cut', 'color', 'clarity'], drop_first=True)
X_encoding

,carat,super_feature,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_E,color_F,color_G,color_H,color_I,color_J,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,3619.2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
1,1.20,3648.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
2,0.30,3575.4,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,1.20,3568.2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
4,1.05,3682.8,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,0.33,3460.8,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
40451,0.32,3449.6,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
40452,0.28,3432.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
40453,1.02,3596.0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0


In [213]:
def myEncoding(df):
    return pd.get_dummies(df, columns=['cut', 'color', 'clarity'], drop_first=True)

Scaling
==

In [214]:
def scaling(df):
    
    scaler = StandardScaler()
    scaling = scaler.fit_transform(df)
    scaled_df = pd.DataFrame(scaling)
    
    return scaled_df

In [216]:
x_scaling = scaling(X_encoding)

Train, Test, Split
==

In [218]:
X_train, X_test, y_train, y_test = train_test_split(x_scaling, y, test_size=0.2, random_state=42)
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}, y_train: {y_train.shape}, y_test: {y_test.shape}")

X_train: (32364, 19), X_test: (8091, 19), y_train: (32364,), y_test: (8091,)


In [219]:
#regressor = LinearRegression()
regressor = RandomForestRegressor()
hyperparameters = regressor.get_params()
regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Ground truth target:', y_test, '\n')
print('Predicted target:', y_pred, '\n')

Model: RandomForestRegressor() 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 1.0, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False} 

Ground truth target: 17775    4851
13506     431
4325     4407
37870    3722
21321    2093
         ... 
3781     3660
26959    1937
15529    1871
36333    5976
31606    4099
Name: price, Length: 8091, dtype: int64 

Predicted target: [4657.59    475.5    4524.05   ... 1937.08   5968.7775 3863.85  ] 



Predictions
==

In [220]:

d_test['super_feature'] = super_feature(d_test)
d_test = d_test[['cut', 'color', 'clarity', 'carat', 'super_feature']]
#d_test = d_test.drop(columns='city')
x_pred = myEncoding(d_test)
x_pred = scaling(x_pred)

In [221]:
d_predictions = regressor.predict(x_pred).clip(0, 30000)
d_predictions

array([ 3177.04      ,  5365.16      , 10260.74      , ...,
        3342.80416667,  2288.71      ,   858.97      ])

In [222]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rmse

644.2346774241735

In [223]:

predictions = pd.DataFrame(d_predictions).rename(columns={0:'price'})
predictions.index.names = ['id']
predictions

,price
id,
0,3177.040000
1,5365.160000
2,10260.740000
3,3992.490000
4,1645.331810
...,...
13480,1634.905000
13481,2537.826000
13482,3342.804167


In [224]:
predictions.to_csv('../results/predictions.csv')

In [225]:
r2 = r2_score(y_test, y_pred)
r2

0.9731169234613943